# Challenge Data Science

Desafio [Alura](https://www.alura.com.br/) -  Um projeto de Data Science com PySpark

![title](resources/home_insigthplaces.png)

A imobiliária **InsightPlaces**, situada na cidade do Rio de Janeiro, está enfrentando dificuldades para alugar e vender imóveis. Em uma pesquisa de como empresas semelhantes operam no mercado, a InsightPlaces percebeu que esse problema pode estar relacionado aos valores dos imóveis e às recomendações que faz.

Sou do time de Ciência de Dados e **Big Data** da InsightPlaces e fiquei responsável por auxiliar no processo de análise de dados dos imóveis localizados em alguns bairros da cidade do Rio de Janeiro.

Dentro desse contexto, como podemos definir de forma eficiente os preços dos imóveis lidando com grande volume de dados? É importante recomendar imóveis utilizando outro critério? O que precisa ser feito?

Esse projeto tem algumas etapas como: ler e fazer o tratamento do histórico dos preços de imóveis no Rio de Janeiro, construir um modelo de regressão para precificar imóveis e criar um recomendador de imóveis. Para cada uma dessas etapas vamos utilizar a ferramenta **PySpark** que oferece uma melhor performance ao trabalharmos com grandes volumes de dado

Vamos que vamos!


##  Semana 01

## Transformando dados com PySpark
Vamos explorar e tratar uma base de dados que veio dos sistemas internos da nossa empresa.

### Imports e carregamento dos Dados

In [46]:
from pyspark.sql import(
    SparkSession, 
    functions as F,
    DataFrame
) 
import utils

In [47]:
spark = (
    SparkSession
    .builder
    .appName('InsightPlaces')
    .getOrCreate()
)

spark

In [48]:
data = (
    spark.read.format('json')
    .option('inferSchema', True)
    .load('data/dataset_bruto.json')
)


### Exploração dos Dados

In [49]:
data.limit(5).show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
+--------------------+--------------------+--------------------+



In [50]:

print('Número de linhas e colunas:', (data.count(), len(data.columns)) )
utils.printSeparator()
data.printSchema()

Número de linhas e colunas: (89083, 3)
root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-

### Análise e Tratamento dos Dados

In [51]:

data.select('anuncio.*', 'imagens', 'usuario.*').limit(5)

andar,area_total,area_util,banheiros,caracteristicas,endereco,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,valores,imagens,id,nome
0,[],[16],[0],[],"{Centro, 20061003...",47d553e0-79f2-4a4...,[0],[0],Usado,Outros,Comercial,[1],"[{260, 107, Venda...",[{39d6282a-71f3-4...,9d44563d-3405-4e8...,Frank
0,[],[14],[0],[],"{Centro, 20051040...",b6ffbae1-17f6-487...,[0],[],Usado,Outros,Comercial,[0],"[{260, 107, Venda...",[{23d2b3ab-45b0-4...,36245be7-70fe-40c...,Caroline
0,[1026],[1026],[0],[],"{Maria da Graça, ...",1fb030a5-9e3e-4a1...,[0],[],Usado,Outros,Comercial,[0],"[{null, 1613, Ven...",[{1da65baa-368b-4...,9dc415d8-1397-4d8...,Oliver
0,[120],[120],[0],[Portão eletrônic...,"{Campo Grande, 23...",ac6cb348-69d6-45a...,[0],[],Usado,Outros,Residencial,[],"[{80, null, Venda...",[{79b542c6-49b4-4...,9911a2df-f299-4a7...,Matthew
0,[3],[3],[0],[],"{São Cristóvão, 2...",e032b908-ef42-4d4...,[0],[],Usado,Outros,Residencial,[],"[{0, 0, Venda, 50...",[{e2bc497b-6510-4...,240a7aab-12e5-406...,Jude


In [52]:
# Explode nas colunas anuncio, usuario e salvando em novo df

data_in_prepared = data.select(
    'anuncio.andar',
    'anuncio.area_total',
    'anuncio.area_util',
    'anuncio.banheiros',
    'anuncio.caracteristicas',
    'anuncio.endereco.*',
    F.col('anuncio.id').alias('id_anuncio'),
    'anuncio.quartos',
    'anuncio.suites',
    'anuncio.tipo_anuncio',
    'anuncio.tipo_unidade',
    'anuncio.tipo_uso',
    'anuncio.vaga',
    'anuncio.valores.condominio',
    'anuncio.valores.iptu',
    'anuncio.valores.tipo',
    'anuncio.valores.valor',
    'imagens',
    'usuario.*'
 )
 
data_in_prepared.limit(3)

# +-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+--------+
# |andar|area_total|area_util|banheiros|caracteristicas                        |endereco                                                                                                                     |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |imagens                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |id                                  |nome    |
# +-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+--------+
# |0    |[]        |[16]     |[0]      |[]                                     |{Centro, 20061003, Rio de Janeiro, Rio de Janeiro, -22.906082, -43.18671, BR, Rua Buenos Aires, Zona Central}                |47d553e0-79f2-4a46-9390-5a3c962740c2|[0]    |[0]   |Usado       |Outros      |Comercial  |[1] |[{260, 107, Venda, 10000}]  |[{39d6282a-71f3-47bc-94aa-909351ecd881, https://api.images.insightplaces.com.br/{type}/{width}/{height}/1b08098d-f2dc-439c-b604-a4e50d12774b.jpg}]                                                                                                                                                                                                                                                                                                                                                                                                                                                      |9d44563d-3405-4e84-9381-35b7cf40a9a4|Frank   |
# |0    |[]        |[14]     |[0]      |[]                                     |{Centro, 20051040, Rio de Janeiro, Rio de Janeiro, -22.902536, -43.18434, BR, Avenida Passos, Zona Central}                  |b6ffbae1-17f6-4870-9950-e998ac1c8d6a|[0]    |[]    |Usado       |Outros      |Comercial  |[0] |[{260, 107, Venda, 10000}]  |[{23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https://api.images.insightplaces.com.br/{type}/{width}/{height}/7a65b929-fe46-4a60-ae31-81faf2edb3c6.jpg}, {1b5f71f8-1aec-4b82-8d4c-67540d204a59, https://api.images.insightplaces.com.br/{type}/{width}/{height}/82e92d72-002a-4d3b-8063-16d4d65474b3.jpg}, {9bb11688-7dc9-48b9-bc1d-6a7a2dbcd553, https://api.images.insightplaces.com.br/{type}/{width}/{height}/23be0c2c-4c8f-4718-8126-6f160bb3efcc.jpg}, {1aaf6095-70bc-4f1f-87a4-fe3534a1bd2a, https://api.images.insightplaces.com.br/{type}/{width}/{height}/0ae382f4-b1ac-4ad9-b6cf-d9b4d7d01908.jpg}]|36245be7-70fe-40cd-84d4-70a57dc8f976|Caroline|
# |0    |[1026]    |[1026]   |[0]      |[]                                     |{Maria da Graça, 20785360, Rio de Janeiro, Rio de Janeiro, -22.885516, -43.269083, BR, Rua Luís de Brito, Zona Norte}        |1fb030a5-9e3e-4a1d-93cd-cd2d8a215e11|[0]    |[]    |Usado       |Outros      |Comercial  |[0] |[{null, 1613, Venda, 10000}]|[{1da65baa-368b-490f-bef7-1834b9a428c1, https://api.images.insightplaces.com.br/{type}/{width}/{height}/b06fe423-6f18-43bb-b0e3-a569158885d2.jpg}]                                                                                                                                                                                                                                                                                                                                                                                                                                                      |9dc415d8-1397-4d8d-8425-b8310fad309d|Oliver  |
# |0    |[120]     |[120]    |[0]      |[Portão eletrônico, Condomínio fechado]|{Campo Grande, 23095071, Rio de Janeiro, Rio de Janeiro, -22.885028, -43.561668, BR, Estrada Guandu do Sape, Zona Oeste}     |ac6cb348-69d6-45af-9589-dc34099370d8|[0]    |[]    |Usado       |Outros      |Residencial|[]  |[{80, null, Venda, 10000}]  |[{79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https://api.images.insightplaces.com.br/{type}/{width}/{height}/b9688b1a-e6e8-4027-9368-d8298bc5ac40.jpg}]                                                                                                                                                                                                                                                                                                                                                                                                                                                      |9911a2df-f299-4a73-a384-281417abaf69|Matthew |
# |0    |[3]       |[3]      |[0]      |[]                                     |{São Cristóvão, 20931670, Rio de Janeiro, Rio de Janeiro, -22.885912, -43.219296, BR, Rua Monsenhor Manuel Gomes, Zona Norte}|e032b908-ef42-4d4a-8125-eba4792bacbe|[0]    |[]    |Usado       |Outros      |Residencial|[]  |[{0, 0, Venda, 5000}]       |[{e2bc497b-6510-4731-8942-97b3d3cdab5e, https://api.images.insightplaces.com.br/{type}/{width}/{height}/bd7e2c54-8d10-4148-806d-0d656ff80fc2.jpg}]                                                                                                                                                                                                                                                                                                                                                                                                                                                      |240a7aab-12e5-4069-9a2c-9dca4c4f9c68|Jude    |
# +-----+----------+---------+---------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+--------+

andar,area_total,area_util,banheiros,caracteristicas,bairro,cep,cidade,estado,latitude,longitude,pais,rua,zona,id_anuncio,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,condominio,iptu,tipo,valor,imagens,id,nome
0,[],[16],[0],[],Centro,20061003,Rio de Janeiro,Rio de Janeiro,-22.906082,-43.18671,BR,Rua Buenos Aires,Zona Central,47d553e0-79f2-4a4...,[0],[0],Usado,Outros,Comercial,[1],[260],[107],[Venda],[10000],[{39d6282a-71f3-4...,9d44563d-3405-4e8...,Frank
0,[],[14],[0],[],Centro,20051040,Rio de Janeiro,Rio de Janeiro,-22.902536,-43.18434,BR,Avenida Passos,Zona Central,b6ffbae1-17f6-487...,[0],[],Usado,Outros,Comercial,[0],[260],[107],[Venda],[10000],[{23d2b3ab-45b0-4...,36245be7-70fe-40c...,Caroline
0,[1026],[1026],[0],[],Maria da Graça,20785360,Rio de Janeiro,Rio de Janeiro,-22.885516,-43.269083,BR,Rua Luís de Brito,Zona Norte,1fb030a5-9e3e-4a1...,[0],[],Usado,Outros,Comercial,[0],[null],[1613],[Venda],[10000],[{1da65baa-368b-4...,9dc415d8-1397-4d8...,Oliver


In [53]:
# Convertendo os tipos de dados

data_in_prepared = (
    data_in_prepared
    .withColumn('area_total', F.concat_ws(',', F.col('area_total')).cast('double') )
    .withColumn('area_util', F.concat_ws(',', F.col('area_util')).cast('double') )
    .withColumn('banheiros', F.concat_ws(',', F.col('banheiros')).cast('double') )
    .withColumn('caracteristicas', F.concat_ws(' - ', F.col('caracteristicas')) )
    .withColumn('quartos', F.concat_ws(',', F.col('quartos')).cast('double') )
    .withColumn('suites', F.concat_ws(',', F.col('suites')).cast('double') )
    .withColumn('vaga', F.concat_ws(',', F.col('vaga')).cast('double') )
    .withColumn('condominio', F.concat_ws(',', F.col('condominio')).cast('double') )
    .withColumn('iptu', F.concat_ws(',', F.col('iptu')).cast('double') )
    .withColumn('tipo', F.concat_ws(',', F.col('tipo')) )
    .withColumn('valor', F.concat_ws(',', F.col('valor')).cast('double') )

)

data_in_prepared.limit(5)

andar,area_total,area_util,banheiros,caracteristicas,bairro,cep,cidade,estado,latitude,longitude,pais,rua,zona,id_anuncio,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,condominio,iptu,tipo,valor,imagens,id,nome
0,null,16.0,0.0,,Centro,20061003,Rio de Janeiro,Rio de Janeiro,-22.906082,-43.18671,BR,Rua Buenos Aires,Zona Central,47d553e0-79f2-4a4...,0.0,0.0,Usado,Outros,Comercial,1.0,260.0,107.0,Venda,10000.0,[{39d6282a-71f3-4...,9d44563d-3405-4e8...,Frank
0,null,14.0,0.0,,Centro,20051040,Rio de Janeiro,Rio de Janeiro,-22.902536,-43.18434,BR,Avenida Passos,Zona Central,b6ffbae1-17f6-487...,0.0,null,Usado,Outros,Comercial,0.0,260.0,107.0,Venda,10000.0,[{23d2b3ab-45b0-4...,36245be7-70fe-40c...,Caroline
0,1026.0,1026.0,0.0,,Maria da Graça,20785360,Rio de Janeiro,Rio de Janeiro,-22.885516,-43.269083,BR,Rua Luís de Brito,Zona Norte,1fb030a5-9e3e-4a1...,0.0,null,Usado,Outros,Comercial,0.0,null,1613.0,Venda,10000.0,[{1da65baa-368b-4...,9dc415d8-1397-4d8...,Oliver
0,120.0,120.0,0.0,Portão eletrônico...,Campo Grande,23095071,Rio de Janeiro,Rio de Janeiro,-22.885028,-43.561668,BR,Estrada Guandu do...,Zona Oeste,ac6cb348-69d6-45a...,0.0,null,Usado,Outros,Residencial,null,80.0,null,Venda,10000.0,[{79b542c6-49b4-4...,9911a2df-f299-4a7...,Matthew
0,3.0,3.0,0.0,,São Cristóvão,20931670,Rio de Janeiro,Rio de Janeiro,-22.885912,-43.219296,BR,Rua Monsenhor Man...,Zona Norte,e032b908-ef42-4d4...,0.0,null,Usado,Outros,Residencial,null,0.0,0.0,Venda,5000.0,[{e2bc497b-6510-4...,240a7aab-12e5-406...,Jude


In [54]:
data_in_prepared.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: double (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: double (nullable = true)
 |-- caracteristicas: string (nullable = false)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- pais: string (nullable = true)
 |-- rua: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id_anuncio: string (nullable = true)
 |-- quartos: double (nullable = true)
 |-- suites: double (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: double (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = false)
 |-- valor: double (nullable = true)
 |-- imagens: ar

### Salvando a base em formato Parquet

In [55]:
data_in_prepared.write.mode('overwrite').parquet('./data/data_in_prepared')

### Lendo a base em formato Parquet

In [56]:
# Novo inicio de partida para as analises

data_in_prepared = (
    spark.read.format('parquet')
    .load('data/data_in_prepared/')
)

O time de Data Science solicitou que fizéssemos alguns filtros nas colunas tipo_uso, tipo_unidade e tipo_anuncio da nossa base de dados:

- tipo_uso: Residencial;
- tipo_unidade: Apartamento;
- tipo_anuncio: Usado.

Verificando as frequências das colunas das colunas selecionadas antes de filtrá-las.

In [57]:
colunas_frequencia = ['tipo_uso', 'tipo_unidade', 'tipo_anuncio']

In [58]:
data_select_tmp = (
    data_in_prepared.select(colunas_frequencia)
)

In [59]:
for c in colunas_frequencia:
    print(data_in_prepared.groupBy(c).count())


+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [60]:
def count_unique_records(df: DataFrame)->None:
    df.agg(*(F.countDistinct(F.col(c)).alias(c) for c in df.columns)).show()

In [61]:
count_unique_records(data_select_tmp)

+--------+------------+------------+
|tipo_uso|tipo_unidade|tipo_anuncio|
+--------+------------+------------+
|       2|           3|           2|
+--------+------------+------------+



In [62]:
# Desconsiderando a variável temporário

data_select_tmp = None

In [63]:
# Efetuando o Filtro

data_in_prepared = (
    data_in_prepared.where(
        (data_in_prepared.tipo_uso == 'Residencial') 
        & (data_in_prepared.tipo_unidade == 'Apartamento')
        & (data_in_prepared.tipo_anuncio == 'Usado')
    )
)

In [64]:
'''A equipe de ciência de dados nos solicitou que apenas as informações 
sobre bairro e zona da cidade fossem extraídas.

Então, vamos analisar a coluna endereço e transformar apenas as 
informações sobre bairro e zona em colunas de nosso DataFrame.
'''

data_in_prepared = (
    data_in_prepared
    .drop('cep')
    .drop('cidade')
    .drop('estado')
    .drop('latitude')
    .drop('longitude')
    .drop('pais')
    .drop('rua')
)

In [66]:
data_in_prepared.limit(10)

andar,area_total,area_util,banheiros,caracteristicas,bairro,zona,id_anuncio,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,condominio,iptu,tipo,valor,imagens,id,nome
3,43.0,43.0,1.0,Academia - Churra...,Paciência,Zona Oeste,d2e3a3aa-09b5-45a...,2.0,null,Usado,Apartamento,Residencial,1.0,245.0,null,Venda,15000.0,[{147a80d9-cd40-4...,5a9736b5-aaa0-4a3...,Hope
2,42.0,42.0,1.0,Churrasqueira - P...,Paciência,Zona Oeste,085bab2c-87ad-452...,2.0,null,Usado,Apartamento,Residencial,1.0,0.0,0.0,Venda,15000.0,[{35740004-063d-4...,ec48d96a-137c-49d...,Margareth
1,41.0,41.0,1.0,Portaria 24h - Co...,Guaratiba,Zona Oeste,18d22cbe-1b86-476...,2.0,null,Usado,Apartamento,Residencial,1.0,0.0,0.0,Venda,20000.0,[{3d404069-418e-4...,a845f35f-3ab3-461...,Nicole
3,43.0,43.0,1.0,Churrasqueira - P...,Cosmos,Zona Oeste,bed8a354-9317-442...,2.0,null,Usado,Apartamento,Residencial,0.0,285.0,null,Venda,20000.0,[{5b8a8de7-2088-4...,8c8de313-45d5-4cc...,Bradley
2,43.0,43.0,1.0,Academia - Churra...,Guaratiba,Zona Oeste,12a13315-d67f-48f...,2.0,null,Usado,Apartamento,Residencial,1.0,245.0,null,Venda,15000.0,[{3fe1c1cc-7a9c-4...,7f2de69f-68f8-404...,Duncan
3,43.0,43.0,1.0,Churrasqueira - P...,Cosmos,Zona Oeste,a2e6d7a5-0ff0-484...,2.0,0.0,Usado,Apartamento,Residencial,0.0,285.0,null,Venda,20000.0,[{fd04f07d-cfdf-4...,12ce772e-4aac-46b...,Mia
3,43.0,43.0,1.0,Churrasqueira - P...,Paciência,Zona Oeste,a6e3173b-c950-4db...,2.0,null,Usado,Apartamento,Residencial,1.0,250.0,null,Venda,15000.0,[{76f97b59-a61b-4...,cc17fc2c-6be2-4e5...,Brooke
3,43.0,43.0,1.0,Churrasqueira - P...,Guaratiba,Zona Oeste,2e6e5dfb-206c-496...,2.0,0.0,Usado,Apartamento,Residencial,1.0,245.0,null,Venda,15000.0,[{95d9678e-491f-4...,286649de-3099-4cc...,Declan
3,43.0,43.0,1.0,Churrasqueira - P...,Paciência,Zona Oeste,99f8d0f9-95a4-461...,2.0,0.0,Usado,Apartamento,Residencial,1.0,245.0,null,Venda,15000.0,[{0d1373a1-db17-4...,8ae860f6-a230-4bd...,Natalie
4,43.0,43.0,1.0,Churrasqueira - P...,Guaratiba,Zona Oeste,b3f44c1a-2d50-4d5...,2.0,null,Usado,Apartamento,Residencial,1.0,240.0,null,Venda,17999.0,[{ab0eae8a-5b02-4...,b1e82c1e-f55d-483...,Carly
